In [2]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# from dotenv import load_dotenv
# import os

In [4]:
import asyncio
from pathlib import Path

import neo4j
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.experimental.pipeline.pipeline import PipelineResult
from neo4j_graphrag.llm import LLMInterface
from neo4j_graphrag.llm import OpenAILLM

In [ ]:
URI = ""
AUTH = ("neo4j", "")
DATABASE = "neo4j"


In [6]:
file_path = "Harry Potter and the Chamber of Secrets Summary.pdf"

In [7]:
ENTITIES = ["Person", "Organization", "Location"]
RELATIONS = ["SITUATED_AT", "INTERACTS", "LED_BY"]
POTENTIAL_SCHEMA = [
    ("Person", "SITUATED_AT", "Location"),
    ("Person", "INTERACTS", "Person"),
    ("Organization", "LED_BY", "Person"),
]

In [ ]:
llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "max_tokens": 2000,
        "response_format": {"type": "json_object"},
    },
)

In [9]:
neo4j_driver=neo4j.GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
kg_builder = SimpleKGPipeline(
        llm=llm,
        driver=neo4j_driver,
        embedder=OpenAIEmbeddings(),
        entities=ENTITIES,
        relations=RELATIONS,
        potential_schema=POTENTIAL_SCHEMA,
        neo4j_database=DATABASE,
    )

In [11]:
res = await kg_builder.run_async(file_path=str(file_path))
print(res)

run_id='e07387c8-78c5-4a73-8b58-36b18c5de824' result={'resolver': {'number_of_nodes_to_resolve': 34, 'number_of_created_nodes': 27}}


In [12]:
import asyncio
import logging

import neo4j
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.experimental.pipeline.pipeline import PipelineResult
from neo4j_graphrag.experimental.pipeline.types.schema import (
    EntityInputType,
    RelationInputType,
)
from neo4j_graphrag.llm import LLMInterface
from neo4j_graphrag.llm import OpenAILLM

In [16]:
TEXT = """
KCR is the Chief Minister of Telangana. Telangana is a state in India. Infosys has an office in Hyderabad. Hyderabad is the capital of Telangana.
Telangana was formed on June 2, 2014, becoming India's 29th state. Hyderabad, known for its rich history and architecture, was founded in 1591 by Muhammad Quli Qutb Shah. The city is home to landmarks like the Charminar and Golconda Fort.
The state is traversed by major rivers such as the Godavari and Krishna, supporting its predominantly agrarian economy. Telangana is a leading producer of rice, cotton, and mangoes. Additionally, it has a significant IT industry presence, with companies like Infosys and Microsoft operating in Hyderabad.
Telangana is also known for its diverse culture, with festivals like Bathukamma and Bonalu celebrated widely.
"""

In [17]:
ENTITIES = ["Person", "Organization", "City", "State", "Country", "River", "Monument", "Festival"]
RELATIONS = ["LEADS", "LOCATED_IN", "HAS_OFFICE", "CAPITAL_OF", "FOUNDED_BY", "FLOWS_THROUGH", "CELEBRATES"]
POTENTIAL_SCHEMA = [
    ("Person", "LEADS", "State"),
    ("State", "LOCATED_IN", "Country"),
    ("Organization", "HAS_OFFICE", "City"),
    ("City", "CAPITAL_OF", "State"),
    ("City", "FOUNDED_BY", "Person"),
    ("River", "FLOWS_THROUGH", "State"),
    ("State", "CELEBRATES", "Festival"),
]


In [ ]:
text_kg_builder = SimpleKGPipeline(
    llm=llm,
        driver=neo4j_driver,
        embedder=OpenAIEmbeddings(),
        entities=ENTITIES,
        relations=RELATIONS,
        potential_schema=POTENTIAL_SCHEMA,
        neo4j_database=DATABASE,
        from_pdf=False
)

In [20]:
res = await text_kg_builder.run_async(text=TEXT)
print(res)

run_id='81162916-f8ce-421a-bfa0-78413bace6ab' result={'resolver': {'number_of_nodes_to_resolve': 37, 'number_of_created_nodes': 37}}
